In [1]:
import traceback
import urllib3
import xmltodict
import re
import pymongo 

client=pymongo.MongoClient()
db=client.earthquake

def getxml():
    url = 'http://www.ssn.unam.mx/rss/ultimos-sismos.xml'

    http = urllib3.PoolManager()

    response = http.request('GET', url)
    try:
        data = xmltodict.parse(response.data)
    except:
        print('Failed to parse xml from response {}'.format(traceback.format_exc()))
    return data

In [2]:
type(getxml())

collections.OrderedDict

In [3]:
d=getxml()

In [4]:
list(d['rss'])

['@version', '@xmlns:geo', 'channel']

In [5]:
list(d['rss']['channel'])

['title', 'link', 'description', 'item']

In [6]:
d['rss']['channel']['item'][0]

OrderedDict([('title', '4.0, 67 km al NORESTE de  GUAMUCHIL, SIN'),
             ('description',
              '<p>Fecha:2019-10-28 09:08:45 (Hora de M&eacute;xico)<br/>Lat/Lon: 26.04/-107.89<br/>Profundidad: 10.0 km </p>'),
             ('link',
              'http://www2.ssn.unam.mx:8080/jsp/localizacion-de-sismo.jsp?latitud=26.04&longitud=-107.89&prf=10.0 km&ma=4.0&fecha=2019-10-28&hora=09:08:45&loc=67 km al NORESTE de  GUAMUCHIL, SIN         &evento=1'),
             ('geo:lat', '26.04'),
             ('geo:long', '-107.89')])

In [7]:
d['rss']['channel']['item'][0]['title']

'4.0, 67 km al NORESTE de  GUAMUCHIL, SIN'

In [8]:
d['rss']['channel']['item'][0]['geo:lat']

'26.04'

In [9]:
d['rss']['channel']['item'][0]['geo:long']

'-107.89'

In [10]:
d['rss']

OrderedDict([('@version', '2.0'),
             ('@xmlns:geo', 'http://www.w3.org/2003/01/geo/wgs84_pos#'),
             ('channel',
              OrderedDict([('title', 'Últimos sismos registrados por el SSN'),
                           ('link', 'http://www.ssn.unam.mx'),
                           ('description',
                            'Reporte de los últimos sismos en México'),
                           ('item',
                            [OrderedDict([('title',
                                           '4.0, 67 km al NORESTE de  GUAMUCHIL, SIN'),
                                          ('description',
                                           '<p>Fecha:2019-10-28 09:08:45 (Hora de M&eacute;xico)<br/>Lat/Lon: 26.04/-107.89<br/>Profundidad: 10.0 km </p>'),
                                          ('link',
                                           'http://www2.ssn.unam.mx:8080/jsp/localizacion-de-sismo.jsp?latitud=26.04&longitud=-107.89&prf=10.0 km&ma=4.0&fecha=2019-10-28

In [11]:
for i in range(len(d['rss']['channel']['item'])):
    print (d['rss']['channel']['item'][i]['description'])

<p>Fecha:2019-10-28 09:08:45 (Hora de M&eacute;xico)<br/>Lat/Lon: 26.04/-107.89<br/>Profundidad: 10.0 km </p>
<p>Fecha:2019-10-28 08:15:02 (Hora de M&eacute;xico)<br/>Lat/Lon: 14.13/-93.34<br/>Profundidad: 19.0 km </p>
<p>Fecha:2019-10-28 04:59:59 (Hora de M&eacute;xico)<br/>Lat/Lon: 17.9/-95.08<br/>Profundidad: 120.0 km </p>
<p>Fecha:2019-10-28 04:55:15 (Hora de M&eacute;xico)<br/>Lat/Lon: 16.69/-99.42<br/>Profundidad: 7.0 km </p>
<p>Fecha:2019-10-28 04:34:54 (Hora de M&eacute;xico)<br/>Lat/Lon: 16.3/-98.25<br/>Profundidad: 13.0 km </p>
<p>Fecha:2019-10-28 03:56:44 (Hora de M&eacute;xico)<br/>Lat/Lon: 15.82/-93.03<br/>Profundidad: 137.0 km </p>
<p>Fecha:2019-10-28 03:30:55 (Hora de M&eacute;xico)<br/>Lat/Lon: 15.55/-98.64<br/>Profundidad: 12.0 km </p>
<p>Fecha:2019-10-28 03:28:58 (Hora de M&eacute;xico)<br/>Lat/Lon: 15.63/-92.8<br/>Profundidad: 144.0 km </p>
<p>Fecha:2019-10-28 03:21:22 (Hora de M&eacute;xico)<br/>Lat/Lon: 25.61/-104.84<br/>Profundidad: 17.0 km </p>
<p>Fecha:2019-10-2

In [12]:
d['rss']['channel']['item'][0]['description']

'<p>Fecha:2019-10-28 09:08:45 (Hora de M&eacute;xico)<br/>Lat/Lon: 26.04/-107.89<br/>Profundidad: 10.0 km </p>'

In [13]:
data_get=d['rss']['channel']['item'][0]['description'].split('>')[1:4]

In [14]:
data_get

['Fecha:2019-10-28 09:08:45 (Hora de M&eacute;xico)<br/',
 'Lat/Lon: 26.04/-107.89<br/',
 'Profundidad: 10.0 km </p']

In [15]:
fecha=re.findall('\d+-\d+-\d+', data_get[0])[0]

In [16]:
hora=re.findall('\d+:\d+:\d+', data_get[0])[0]

In [17]:
lat=re.findall('-?\d+.\d+', data_get[1])[0]

In [18]:
lon=re.findall('-?\d+.\d+', data_get[1])[1]

In [19]:
prof=re.findall('\d+.\d+ \w+', data_get[2])[0]

In [20]:
data={'Fecha':fecha, 'Hora':hora, 'Latitud':float(lat), 'Longitud':float(lon), 'Profundidad':prof}

In [21]:
data

{'Fecha': '2019-10-28',
 'Hora': '09:08:45',
 'Latitud': 26.04,
 'Longitud': -107.89,
 'Profundidad': '10.0 km'}

In [22]:
todo=list(db.ssnUnam.find())
#print (todo)
for i in range(len(d['rss']['channel']['item'])):
    data_get=d['rss']['channel']['item'][i]['description'].split('>')[1:4]
    fecha=re.findall('\d+-\d+-\d+', data_get[0])[0]
    hora=re.findall('\d+:\d+:\d+', data_get[0])[0]
    lat=re.findall('-?\d+.\d+', data_get[1])[0]
    lon=re.findall('-?\d+.\d+', data_get[1])[1]
    prof=re.findall('\d+.\d+ \w+', data_get[2])[0]
    data={'Fecha':fecha, 'Hora':hora,
          'Latitud':float(lat), 'Longitud':float(lon),
          'Profundidad':prof}
    
    db.ssnUnam.update(data, data, upsert=True)

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


In [25]:
d['rss']['channel']['item'][0]['title'].split(',')[0]

'4.0'